# Compare model results with stations

Load the SMRF and iSnobal results to plot against the stations


In [1]:
import os
from glob import glob

import ipywidgets as widgets
import pandas as pd 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
# import pandas_bokeh
# pandas_bokeh.output_notebook()


smrf_dir = '/data/output/johnston_draw/devel/wy2009/jd_3meter/data/data0024_8711/smrfOutputs/'

In [2]:
da = xr.open_mfdataset(os.path.join(smrf_dir, '*.nc'))


In [3]:
# load the station data and pull out variables

metadata = pd.read_csv('../station_data/metadata.csv')
metadata.set_index('primary_id', inplace=True)

df = []
for file_name in glob('../station_data/*.csv'):
    variable = os.path.splitext(os.path.basename(file_name))[0]
    if variable != 'metadata':
        dd = pd.read_csv(file_name, parse_dates=['date_time'])
        dd['variable'] = variable
        df.append(dd)

df = pd.concat(df).reset_index().set_index(['variable', 'date_time']).sort_index()
del df['index']
df

jd144  jd163  jd124b  jd125  jd124  jdt1  \
variable   date_time                                                       
air_temp   2008-10-01 00:00:00    NaN    NaN    21.6   23.5   21.1  22.5   
           2008-10-01 01:00:00    NaN    NaN    20.1   21.6   20.5  21.0   
           2008-10-01 02:00:00    NaN    NaN    19.6   20.2   19.6  20.9   
           2008-10-01 03:00:00    NaN    NaN    19.6   21.1   19.3  20.6   
           2008-10-01 04:00:00    NaN    NaN    18.4   20.1   18.3  20.1   
...                               ...    ...     ...    ...    ...   ...   
wind_speed 2009-09-30 18:00:00    NaN    NaN     2.6    4.5   11.5   NaN   
           2009-09-30 19:00:00    NaN    NaN     2.5    4.1   10.5   NaN   
           2009-09-30 20:00:00    NaN    NaN     2.4    3.5    9.0   NaN   
           2009-09-30 21:00:00    NaN    NaN     2.5    2.8    8.2   NaN   
           2009-09-30 22:00:00    NaN    NaN     2.1    2.6    8.5   NaN   

                                jdt2  jdt2b  jdt3  jdt3b  jdt4  jdt4b  jdt5  
variable   date_time                                                         
air_temp   2008-10-01 00:00:00  21.9    NaN  21.7    NaN  21.4    NaN  21.1  
           2008-10-01 01:00:00  20.9    NaN  20.7    NaN  20.4    NaN  19.6  
           2008-10-01 02:00:00  20.2    NaN  19.5    NaN  20.2    NaN  20.3  
           2008-10-01 03:00:00  20.1    NaN  19.7    NaN  19.4    NaN  19.1  
           2008-10-01 04:00:00  19.7    NaN  19.0    NaN  19.1    NaN  19.1  
...                              ...    ...   ...    ...   ...    ...   ...  
wind_speed 2009-09-30 18:00:00   NaN    NaN   NaN    NaN   NaN    NaN   NaN  
           2009-09-30 19:00:00   NaN    NaN   NaN    NaN   NaN    NaN   NaN  
           2009-09-30 20:00:00   NaN    NaN   NaN    NaN   NaN    NaN   NaN  
           2009-09-30 21:00:00   NaN    NaN   NaN    NaN   NaN    NaN   NaN  
           2009-09-30 22:00:00   NaN    NaN   NaN    NaN   NaN    NaN   NaN  

[61290 rows x 13 columns]

In [ ]:
# load the SMRF and model results

station = 'jd124b'
variables = ['air_temp', 'vapor_pressure']

smrf_df = []

for variable in da.keys():
    
    if da[variable].size > 10:
    
        df_var = []
        for station in metadata.index:
            dd = da[variable].sel(x=metadata.loc[station, 'X'], y=metadata.loc[station, 'Y'], method='nearest').to_dataframe(name=station)
            dd = dd[station].to_frame()
            df_var.append(dd)

        df_var = pd.concat(df_var, axis=1)
        df_var['variable'] = variable

        smrf_df.append(df_var)
    
smrf_df = pd.concat(smrf_df).reset_index().set_index(['variable', 'time']).sort_index()

smrf_df


In [ ]:
# plot the time series data
station_dropdown = widgets.Dropdown(
    options=df.columns,
    description='Station: ',
)

variable_dropdown = widgets.Dropdown(
    options=df.index.unique(level=0),
    description='Variable: '
)

# dates = df.index.unique(level=1)
# index = (0, len(dates)-1)
# selection_range_slider = widgets.SelectionRangeSlider(
#     options=dates,
#     index=index,
#     description='Dates',
#     orientation='horizontal',
#     layout={'width': '1000px'}
# )
# selection_range_slider

def plot_data(variable, station):
    
    plot_df = pd.concat([
        smrf_df.loc[(variable,), station]
    ])


widgets.interact(
    plot_data,
    variable=variable_dropdown,
    station=station_dropdown
)

In [ ]:
# plot of the data with time slider
times = list(da.time.dt.strftime("%Y-%m-%d %H:%M:%S").values)

date_select_slider = widgets.SelectionSlider(
    options=times,
    layout={'width': '500px'},
    orientation='horizontal',
    readout=True
)

variable_dropdown = widgets.Dropdown(
    options=list(da.keys())
)

def display_data(variable, date_time):
    da.sel(time=date_time)[variable].plot(
#         vmax=15,
        figsize=(20,10),
        aspect='equal'
    )

%matplotlib inline
widgets.interact(
    display_data,
    variable=variable_dropdown,
    date_time=date_select_slider
)

In [8]:
# cumulative precip
ppt = da.precip.sum(axis=0)
ppt.values

KeyboardInterrupt: 

In [7]:
ppt.plot()

KeyboardInterrupt: 